In [ ]:
from __future__ import print_function

import audioop
from ctypes import *
import RPi.GPIO as GPIO
import ktkws # KWS
import MicrophoneStream as MS
KWSID = ['기가지니', '지니야', '친구야', '자기야']
RATE = 16000
CHUNK = 512
import grpc
import gigagenieRPC_pb2
import gigagenieRPC_pb2_grpc
import MicrophoneStream as MS
import user_auth as UA
import audioop
import os
from ctypes import *

HOST = 'gate.gigagenie.ai'
PORT = 4080
RATE = 16000
CHUNK = 512
import grpc
import gigagenieRPC_pb2
import gigagenieRPC_pb2_grpc
import user_auth as UA
import os

HOST = 'gate.gigagenie.ai'
PORT = 4080
def getText2VoiceUrl(inText):

	channel = grpc.secure_channel('{}:{}'.format(HOST, PORT), UA.getCredentials())
	stub = gigagenieRPC_pb2_grpc.GigagenieStub(channel)

	message = gigagenieRPC_pb2.reqText()
	message.lang=0
	message.mode=0
	message.text=inText
	response = stub.getText2VoiceUrl(message)

	print ("\n\nresultCd: %d" % (response.resultCd))
	if response.resultCd == 200:
		print ("TTS 생성에 성공하였습니다.\n\n\n아래 URL을 웹브라우져에 넣어보세요.")
		print ("Stream Url: %s\n\n" % (response.url))
	else:
		print ("TTS 생성에 실패하였습니다.")
		print ("Fail: %d" % (response.resultCd)) 

def main():
	getText2VoiceUrl("안녕하세요. 반갑습니다.")

if __name__ == '__main__':
	main()
def queryByText(text):

	channel = grpc.secure_channel('{}:{}'.format(HOST, PORT), UA.getCredentials())
	stub = gigagenieRPC_pb2_grpc.GigagenieStub(channel)

	message = gigagenieRPC_pb2.reqQueryText()
	message.queryText = text
	message.userSession = "1234"
	message.deviceId = "yourdevice"
		
	response = stub.queryByText(message)

	print ("\n\nresultCd: %d" % (response.resultCd))
	if response.resultCd == 200:
		print ("\n\n\n질의한 내용: %s" % (response.uword))
		#dssAction = response.action
		for a in response.action:
			response = a.mesg
		parsing_resp = response.replace('<![CDATA[', '')
		parsing_resp = parsing_resp.replace(']]>', '')
		print("\n\n질의에 대한 답변: " + parsing_resp + '\n\n\n')
		#return response.url
	else:
		print ("Fail: %d" % (response.resultCd))
		#return None	 

def main():

	# Dialog : queryByText
	queryByText("안녕")

if __name__ == '__main__':
	main()
def queryByVoice():
	print ("\n\n\n질의할 내용을 말씀해 보세요.\n\n듣고 있는 중......\n")
	channel = grpc.secure_channel('{}:{}'.format(HOST, PORT), UA.getCredentials())
	stub = gigagenieRPC_pb2_grpc.GigagenieStub(channel)
	request = generate_request()
	resultText = ''
	response = stub.queryByVoice(request)
	if response.resultCd == 200:
		print("질의 내용: %s" % (response.uword))
		for a in response.action:
			response = a.mesg
			parsing_resp = response.replace('<![CDATA[', '')
			parsing_resp = parsing_resp.replace(']]>', '')
			resultText = parsing_resp
			print("\n질의에 대한 답변: " + parsing_resp +'\n\n\n')

	else:
		print("\n\nresultCd: %d\n" % (response.resultCd))
		print("정상적인 음성인식이 되지 않았습니다.")
	return resultText

def main():
	queryByVoice()
	time.sleep(0.5)

if __name__ == '__main__':
	main()
import time
import ex2_getVoice2Text as gv2t
import ex1_kwstest as kws

def main():
	#Example7 KWS+STT

	KWSID = ['기가지니', '지니야', '친구야', '자기야']
	while 1:
		recog=kws.test(KWSID[0])
		if recog == 200:
			print('KWS Dectected ...\n Start STT...')
			text = gv2t.getVoice2Text()
			print('Recognized Text: '+ text)
			time.sleep(2)
			
		else:
			print('KWS Not Dectected ...')

if __name__ == '__main__':
    main()
import MicrophoneStream as MS
import ex1_kwstest as kws
import ex4_getText2VoiceStream as tts
import ex6_queryVoice as dss
import time

def main():
	#Example8 KWS+STT+DSS

	KWSID = ['기가지니', '지니야', '친구야', '자기야']
	while 1:
		recog=kws.test(KWSID[0])
		if recog == 200:
			print('KWS Dectected ...\n')
			dss_answer = dss.queryByVoice()
			tts_result = tts.getText2VoiceStream(dss_answer, "result_mesg.wav")
			if dss_answer == '':
				print('질의한 내용이 없습니다.\n\n\n')
			elif tts_result == 500:
				print("TTS 동작 에러입니다.\n\n\n")
				break
			else:
				MS.play_file("result_mesg.wav")			
			time.sleep(2)
		else:
			print('KWS Not Dectected ...')

if __name__ == '__main__':
    main()
    from __future__ import print_function
import ex1_kwstest as kws
import ex4_getText2VoiceStream as tts
import ex6_queryVoice as dss
import MicrophoneStream as MS
import time
	
def main():
	#Example8 Button+STT+DSS
	KWSID = ['기가지니', '지니야', '친구야', '자기야']
	while 1:
		recog = kws.btn_test(KWSID[0])
		if recog == 200:
			print('KWS Dectected ...\n Start STT...')
			text = dss.queryByVoice()
			tts_result = tts.getText2VoiceStream(text, "result_mesg.wav")
			if text == '':
				print('질의한 내용이 없습니다.')
			elif tts_result == 500:
				print("TTS 동작 에러입니다.\n")
				break
			else:
				MS.play_file("result_mesg.wav")
			time.sleep(2)
		else:
			print('KWS Not Dectected ...')

if __name__ == '__main__':
    main()